In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import optimizers
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input,Flatten,Dense,Conv2D,ReLU,BatchNormalization,MaxPool2D
from keras.optimizers import Adam
from keras.regularizers import l2
%matplotlib inline

# Preprocessing Data

In [ ]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)  
pd.set_option('max_rows', None)
data = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
Y = data['label'].to_numpy()
data = data.drop(columns=['label'])
X = data.to_numpy()
test = test.to_numpy()
X = X.reshape(-1,28,28,1)
test = test.reshape(-1,28,28,1)
X = X/255.0
test = test/255.0
indices = np.random.randint(low=0, high=42000, size=10)
for i in indices:
    plt.figure()
    plt.imshow(X[i,:,:,0], cmap='gray')
Y = to_categorical(Y, 10)

# Splitting the data into Training Set and Testing Set

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4)


# Building a Sequential Model With 5 Convolution layer and 3 Max Pooling Layer


In [ ]:
model = tf.keras.models.Sequential([
     tf.keras.layers.Conv2D(32,(5,5), activation = 'relu', input_shape = (28,28,1)),
     tf.keras.layers.Conv2D(32,(5,5), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
     tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Dropout(0.25),    
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(512, activation='relu'),
     tf.keras.layers.Dense(512, activation='relu'),
     tf.keras.layers.Dense(512, activation='relu'),
     tf.keras.layers.Dense(512, activation='relu'),
     tf.keras.layers.Dropout(0.5),

     tf.keras.layers.Dense(10 , activation = 'softmax')
])

# Model Summary

In [ ]:
model.summary()

# Model Compilation

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Creating Callbacks

In [ ]:
from keras.callbacks import ModelCheckpoint   
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.5, verbose=1,
                              patience=3, min_lr=0.00001)

# Image Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
rotation_range = 45,
width_shift_range= 0.05,
height_shift_range= 0.05,
shear_range = 0.05,
zoom_range = 0.05,
#horizontal_flip = True,
#vertical_flip = True,
fill_mode = 'nearest'
)
train_datagen.fit(X_train)

validation_datagen = ImageDataGenerator(
rotation_range = 45,
width_shift_range= 0.05,
height_shift_range= 0.05,
shear_range = 0.05,
zoom_range = 0.05,
#horizontal_flip = True,
#vertical_flip = True,
fill_mode = 'nearest'
)
validation_datagen.fit(X_test)

# Training The Model

In [ ]:
history = model.fit_generator(train_datagen.flow(X_train, Y_train),epochs=50 ,steps_per_epoch=len(X_train)/32, validation_data=(validation_datagen.fit(X_test),Y_test),callbacks=[reduce_lr],shuffle=True)

# Plotting Loss of the traning model

In [ ]:

width = 12
height = 10
plt.figure(figsize=(width, height))
plt.xlabel('Epoch Number')
plt.ylabel("Loss Magnitude")
plt.plot(history.history['loss'])
plt.show()

# Plotting Accuracy of the traning model

In [ ]:
width = 12
height = 10
plt.figure(figsize=(width, height))
plt.xlabel('Epoch Number')
plt.ylabel("Accuracy Magnitude")
plt.plot(history.history['val_accuracy'])
plt.show()

In [ ]:
pred = model.predict(test)
preds = pred.argmax(axis=1)
imageid = np.arange(1,28001)
output = pd.DataFrame({"ImageId": imageid,"Label": preds})
output.to_csv('submissions_IA1.csv', index=False)

In [ ]:

result= pd.read_csv('../input/digit-recog-score/submissions1 (1).csv')
result=result.drop(columns=['ImageId'],axis=1)


In [ ]:
# DISTRIBUTION 
plt.figure(figsize=(width, height))

ax1 = sns.distplot(preds, hist=False, color="b", label="New Prediction")
sns.distplot(result, hist=False, color="g", label="Previous Prediction" , ax=ax1)



plt.show()
plt.close()